In [ ]:
import os
import configparser

# Loading configurations
configParser = configparser.RawConfigParser()   
configFilePath = r'configuration.txt'
configParser.read(configFilePath)

insert_amd_env_vars =  int(configParser.get('COMMON', 'insert_amd_env_vars'))
HSA_OVERRIDE_GFX_VERSION =  configParser.get('COMMON', 'HSA_OVERRIDE_GFX_VERSION')
ROCM_PATH =  configParser.get('COMMON', 'ROCM_PATH')

if(insert_amd_env_vars != 0):
    os.environ["HSA_OVERRIDE_GFX_VERSION"] = HSA_OVERRIDE_GFX_VERSION
    os.environ["ROCM_PATH"] = ROCM_PATH
    
#import os
#os.environ["TF_CPP_MIN_LOG_LEVEL"] = "2"
#os.environ["AUTOGRAPH_VERBOSITY"] = "0"

In [ ]:

import configparser

# Loading configurations
configParser = configparser.RawConfigParser()   
configFilePath = r'configuration.txt'
configParser.read(configFilePath)


datasetPathGeneratedFaces =  configParser.get('evaluate_imagen', 'test_datasetPathGeneratedFaces')
generated_face_table_name =  configParser.get('evaluate_imagen', 'generated_face_table_name')


datasetPathDatabase =  configParser.get('COMMON', 'test_datasetPathDatabase') + '/dataset.db'



cuda =  int(configParser.get('COMMON', 'cuda'))
cpus =  int(configParser.get('COMMON', 'cpus'))




In [ ]:
#TODO document jupyter
import pickle
import sqlite3 as sl
import pandas as pd
import numpy as np
from PIL import Image

from textwrap import wrap
import os


import matplotlib.pyplot as plt
import numpy as np
import pandas as pd



In [ ]:
pd.set_option('display.max_columns', None)
pd.options.mode.chained_assignment = None


In [ ]:
image_size =  int(configParser.get('COMMON', 'resizeImageTo'))
audio_length_used =  configParser.get('evaluate_imagen', 'audio_length_used') 
model_filename =  configParser.get('evaluate_imagen', 'model_filename') + '_'  + audio_length_used +  's.pt'
openl3_mode =  configParser.get('evaluate_imagen', 'openl3_mode')

In [ ]:
import lancedb
uri = datasetPathDatabase
db = lancedb.connect(uri)

table = db.open_table("video_stage1")

In [ ]:
try:
  db.drop_table(generated_face_table_name)
except:
  print("")

In [ ]:
import multiprocessing
multiprocessing.set_start_method('spawn')

In [ ]:
import pathlib
import os
from random import randrange


uri = datasetPathDatabase
db = lancedb.connect(uri)


files_to_insert = []


for root, dirs, files in os.walk(datasetPathGeneratedFaces):
    for file in files:
        files_to_insert.append(root +'/'+ file)
        #print(root + file)



In [ ]:
audio_folder = output_folder = r'imagen_testing_folder' 
if not os.path.exists(audio_folder):
    os.makedirs(audio_folder)

In [ ]:

from testing_imagen_vision_transformer import extract_vision_transformer
from testing_imagen_face import extract_face_rep
import pathlib
import shutil


In [ ]:


i = 0
for face_file in files_to_insert:
    
    q = multiprocessing.Queue()

    try:
        proc = multiprocessing.Process(target=extract_face_rep, args=(q,face_file,
                                                                image_size,output_folder,))
        proc.start()
        proc.join()
    except:
        continue

    with open(output_folder + '/' + 'vgg_generated_face.pickle', 'rb') as handle:
        vgg = pickle.load(handle)
        #print("++++++" + str(vgg))


    
    tbl = db.open_table("video_stage1")
    result = tbl.search(vgg).limit(5).to_df()

    match_1 = result.iloc[0]['user']
    match_2 = result.iloc[1]['user']
    match_3 = result.iloc[2]['user']
    match_4 = result.iloc[3]['user']
    match_5 = result.iloc[4]['user']

    p_small = pathlib.Path(os.path.dirname(face_file))
    p_big = p_small.parent.absolute()
    
    df_table = pd.DataFrame()
    df_table = df_table.append({'id': i , 
                    'generated_face_path': face_file,'user':p_big.name,
                    'match_1':match_1,'match_2':match_2,'match_3':match_3,'match_4':match_4,'match_5':match_5,
        'vector' : vgg,'stage': 1}, ignore_index=True)
    

    if(i == 0):
        db.create_table(generated_face_table_name, df_table)
    else:
        tbl = db.open_table(generated_face_table_name)
        tbl.add(df_table)


    print(i)
    i = i + 1

shutil.rmtree(output_folder)


In [ ]:
tbl = db.open_table(generated_face_table_name)

In [ ]:
tbl.to_pandas()